In [0]:
# This code was developed using ideas from the following sources:

# DCGAN Adventures with Cifar10 by Stepan Ulyanin
# Deep Convolutional Generative Adversarial Network Tensorflow Tutorial
# How to Develop a GAN to Generate CIFAR10 Small Color Photographs by Jason Brownlee
# Having Fun with Deep Convolutional GANs by Naoki Shibuya

In [0]:
# Import necessary libraries
# Neural net will be constructed using tensorflow.keras

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import numpy as np
from tensorflow import keras
# from tensorflow.keras import layers
from matplotlib import pyplot
import tensorflow_datasets as tfds

In [0]:
# Some helper functions for manipulating MNIST data

# select random real samples to be classified by discriminator
def real_samples(data,n_samples):
  # data is the CIFAR10 dataset
  # take random elements from the dataset
  x = np.random.randint(0,data.shape[0],n_samples)
  X = data[x] #this is the corresponding image
  y = np.ones((n_samples,1)) # this classifies the X value as a real image
  return X , y

# given fake points in latent space, generate some fake images to give to discriminator
def make_fake_samples(generator,lat_dim,n_samples):
  x_in = np.random.randn(n_samples,lat_dim)
  # given the input, the generator model uses the Keras function 'predict' to \\
  #   produce outputs (like our FeedForward technique from class)
  X = generator.predict(x_in)
  # we label these outputs as fake, for classifying with the discriminator
  y = np.zeros((n_samples,1))
  return X , y


In [0]:
# Some more functions for displaying and measuring the network

# save a plot of fake images after a certain number of epochs
def save_plot(images,epoch,n=5):
  # scale images to an appropriate range for imshow:
  images = (images*127.5)+127.5
  images = images.reshape(images.shape[0],28,28)

  for i in range(n**2):
    pyplot.subplot(n,n,i+1); pyplot.axis('off')
    pyplot.imshow(images[i],cmap='gray')
  
  #save the images
  file1 = 'plot_fakes_epoch{}.png'.format(epoch+1)
  pyplot.savefig(file1)
  pyplot.close()



In [0]:
# construct the generator, discriminator, and the full neural net by combining them

# First the discriminator model, which takes in a 32x32x3 image.
#  We use Conv2D to build layers. We can construct Conv2D in a way such that it
#  Reduces the x,y dimensions of the previous layer by half. The parameters 
#  'kernel', 'strides' and 'padding will change the size of the next layer. Literature
#  recommends using kernel = (3,3) for this problem.
#  Example: given an input layer with dimensions (64,64,128).
#           Conv2D(256,(3,3), strides = (2,2)) will give dimensions (62,62,256)
#           Conv2D(128,(3,3), strides = (2,2),padding = 'same') 
#                                      will give dimensions (32,32,128)

def build_discriminator(shape_in = (28,28,1)):
  disc_model = keras.Sequential()
  
  disc_model.add(keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',input_shape=shape_in))

  disc_model.add(keras.layers.LeakyReLU())
  disc_model.add(keras.layers.Dropout(0.3))

  disc_model.add(keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  disc_model.add(keras.layers.LeakyReLU())
  disc_model.add(keras.layers.Dropout(0.3))
  
  disc_model.add(keras.layers.Flatten())
  disc_model.add(keras.layers.Dense(1))
  
  #opt = keras.optimizers.Adam(1e-4)
  #disc_model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

  return disc_model

# next the generator model
# Here we use Conv2DTranspose, and instead of downsampling the dimension of our
#  layers, it upsamples. so we can instead go from (4,4,256) to (8,8,128). The
#  parameters are largely the same otherwise.
def build_generator(lat_dim):
  # lat_dim is the dimension of the latent space (usually lat_dim=100)
  gen_model = keras.Sequential()

  gen_model.add(keras.layers.Dense(7*7*256, use_bias=False, input_dim=lat_dim))
  gen_model.add(keras.layers.BatchNormalization())
  gen_model.add(keras.layers.LeakyReLU())

  gen_model.add(keras.layers.Reshape((7, 7, 256)))

  gen_model.add(keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
  gen_model.add(keras.layers.BatchNormalization())
  gen_model.add(keras.layers.LeakyReLU())

  gen_model.add(keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  gen_model.add(keras.layers.BatchNormalization())
  gen_model.add(keras.layers.LeakyReLU())

  gen_model.add(keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
  
  #opt = keras.optimizers.Adam(lr=0.0004)
  #gen_model.compile(loss='binary_crossentropy',optimizer=opt)

  return gen_model

# now we put them together
def build_dcgan(gen_model,disc_model):
  # don't train weights in discriminator
  #disc_model2 = tf.keras.models.clone_model(disc_model)
  disc_model.trainable = False

  gan_model = keras.Sequential()

  gan_model.add(gen_model)
  gan_model.add(disc_model)

  opt = keras.optimizers.Adam(lr=0.0004) #try lr = 0.0002 or 0.001
  gan_model.compile(loss='binary_crossentropy',optimizer=opt)

  return gan_model




In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [0]:
# now we can train our gan model

def train(generator,discriminator,data,lat_dim,epochs=50):

  for i in range(epochs):
    for images in data:
      noise = tf.random.normal([images.shape[0],lat_dim])

      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        generated_images = generator(noise,training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output,fake_output)

      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    print('epoch = {}, Loss = {}'.format(i+1,disc_loss))
    if (i+1) % 2 == 0:
      n = 5
      latent_points = tf.random.normal([n**2,lat_dim])
      X = (generator.predict(latent_points)) #*127.5)+127.5
      X = X.reshape(N,28,28)    
      save_plot(X,epoch,n)

    

In [0]:
# load the data in from keras.datasets
(data,_),(_,_) = keras.datasets.mnist.load_data()
# Each element of data has dimension 32x32x3
# Convert data to from ints to floats
data = data.astype('float32')
# Map image data from [0,255] to [-1,1]
data = (data-127.5) / 127.5
data = data.reshape(data.shape[0],28,28,1)
train_data = tf.data.Dataset.from_tensor_slices(data).shuffle(data.shape[0]).batch(256)

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# now we will build our model:
lat_dim = 100
disc_model = build_discriminator()
gen_model = build_generator(lat_dim)



In [0]:
#train(gen_model,disc_model,gan_model,data,lat_dim,epochs=60)
train(gen_model,disc_model,train_data,lat_dim,epochs=3)

epoch = 1, Loss = 1.249799370765686
epoch = 2, Loss = 0.7263081669807434


NameError: ignored

In [0]:
percent_correct(10,gen_model,disc_model,data,lat_dim,n_samples=100)

# Uncomment to save the model

#gen_model.compile()
#gen_model.save('generator_mnist.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('generator100e.h5')

4/4 [==============================] - 0s 2ms/step - loss: 0.0000e+00


TypeError: ignored

In [0]:
gen_model.summary()
disc_model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 12544)             1254400   
_________________________________________________________________
batch_normalization_21 (Batc (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_21 (Conv2DT (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_22 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 7, 7, 128)       

In [0]:
N = 5
epoch = 50

latent_points = tf.random.normal([N,100])

X = (gen_model.predict(latent_points)) #*127.5)+127.5
X = X.reshape(N,28,28)

pyplot.figure(figsize = (11,11))
for i in range(N):
  pyplot.subplot(1,N,i+1);
  pyplot.axis('off')
  pyplot.imshow(X[i, :, :],cmap = 'gray');
file1 = 'plot_fakes_epoch{}.png'.format(epoch+1)
pyplot.savefig(file1)
pyplot.close()

In [0]:
latent_points = tf.random.normal([3000,100])

X = (gen_model.predict(latent_points)) #*127.5)+127.5
X = X.reshape(3000,28,28)

np.save('MNIST_fake_images',X)